In [4]:
#configparser
import configparser
global HOST,PORT,DB_ID,DB_PW,MODEL_DIR,MODEL_NAME
conf_dir = '/home/cdsadmin/python_src/EY/Emart/conf/config.ini'
cfg = configparser.ConfigParser(interpolation=configparser.ExtendedInterpolation())
cfg.read(conf_dir)
# default package 
import numpy as np
import pandas as pd 
import time,os,psutil
import IPython
from datetime import datetime
from IPython.lib import kernel
import collections
import itertools
import pickle,ast,joblib
## hana DB client 
from hdbcli import dbapi 
from IPython.core.display import display, Image
import multiprocessing
import sqlite3

#visualization 
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.metrics import plot_confusion_matrix

# scale & eda
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from sklearn.impute import SimpleImputer
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell 
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

#--------------------------------------------------------
# ■ setting configuration (db_connectionm, load_model)
#--------------------------------------------------------
HOST  = cfg['dbconnect']['host']
PORT  = int(cfg['dbconnect']['port'])
DB_ID = cfg['dbconnect']['ID']
DB_PW = cfg['dbconnect']['PW']
MODEL_DIR  = cfg['Mollys']['MODEL_DIR']
#---------------------------------------------------------
InteractiveShell.ast_node_interactivity = "all"

['/home/cdsadmin/python_src/EY/Emart/conf/config.ini']

##### SQL_worker(num) 함수의 where 조건이 현재  PRDT_DI_CD  = 42 로 몰리스 담당 데이터만 가져오게 되어있는데
##### 전체 상품으로 돌려보고싶은경우 해당 조건을 삭제후 로드 

In [8]:
# %%writefile multiprocessing_sequence.py
## --------------------------------------------------
## load_data
## --------------------------------------------------
## multiprocessing 준비

def check_pid():
    connection_file_path = kernel.get_connection_file()
    connection_file      = os.path.basename(connection_file_path)
    kernel_id            = connection_file.split('-', 1)[1].split('.')[0]
    print('■ [PID]          : %s '%os.getpid())
    print('■ [Jupyter path] : %s' %os.getcwd())
    print('■ [Kernel ID]    : %s' %kernel_id)

def timecheck(start):
    learn_time = time.time() - start
    train_time = round(learn_time/60)  
    print("소요시간 :",  round(learn_time,-1), "초", "  (",train_time, "분 )\n")
    return learn_time,train_time
    
def DB_Connection() :
    conn= dbapi.connect(HOST,PORT,DB_ID,DB_PW)
    return conn

def Datachunk_range(rownum_sql) :
    conn = DB_Connection()
    cnt = pd.read_sql(rownum_sql, conn)
    key = int(cnt.values)
    print('Data  count : %s'%key)
    rm  = key % n_core
    ## Split row_num / n_core 
    arg_list_flt =[i for i in range(0,key,int(key/n_core))]
    ## last sequence add remainder 
    if rm > 0:
        arg_list_flt[len(arg_list_flt)-1] = arg_list_flt[len(arg_list_flt)-1]+rm
    else:
        arg_list_flt.append(arg_list_flt[-1]+arg_list_flt[1])
    conn.close()        
    return arg_list_flt,rm

def SQL_worker(num):    
    b = int(arg_list.index(num))-1
    if (int(arg_list.index(num)) == 0) :
        result = None
    else :
        start_num = int(arg_list[b])+1
        end_num   = int(num)
        print('start_num : %s | end_num : %s'%(start_num,end_num))
        sql =f'''
                SELECT *
                FROM (SELECT   ROW_NUMBER() OVER () AS ROW_NUM
                             , CUST_ID
                             , PRDT_CD
                             , PRDT_NM
                             , PRDT_MCODE_CD
                             , PRDT_MCODE_NM
                             , PRDT_DCODE_CD
                             , PRDT_DCODE_NM
                      FROM  {db_name}
                      WHERE PRDT_DI_CD  = 42) T1
                WHERE T1.ROW_NUM between {start_num} and {end_num}
              '''
        conn = DB_Connection()
        result = pd.read_sql(sql, conn)
        conn.close()
    return result

def multiprocesser(arg_list) : 
    check_pid()
    print('\nstart_arg_list : %s \n'%arg_list)
    p = multiprocessing.Pool(processes = n_core)
    data = p.map(SQL_worker, arg_list) 
    p.close()
    p.join()
    result = pd.concat(data)
    return result   

def check_cpu_percent(num):
    cpu_cnt = len(psutil.Process().cpu_affinity())
    c_list = psutil.cpu_percent(percpu=True)
    available_cpu_cnt = len([i for i in c_list if i < 1])
    n_core = np.trunc(available_cpu_cnt/num)
    print('[ total_cpu_cnt ] : ',cpu_cnt)
    print('[ available_cpu_cnt ] : ',available_cpu_cnt)
    return int(n_core)

# 데이터로드 ( multiprocessing n_core = 5)
#### n_core수대로 데이터를 ROW_NUM 수로 Split하여 받아옴) 
#### 예시테이블은 템프성으로 개발서버에 CREATE 시킨 테이블이며 영수증데이터와 코드마스터를 결합해서
#### SKU별 담당코드를 붙인 테이블이 있다면 차후 사용할 때 테이블명만 변경하여 사용가능 

In [11]:
db_name  = 'EY_YSR_MOLLYS_RCIP_DETAIL_BAIN_202008'
# n_core = check_cpu_percent(4)
n_core = 5
print('[n_core] : ',n_core)
cnt_sql  = f''' SELECT COUNT(1)
                FROM {db_name}
                WHERE PRDT_DI_CD  = 42 '''
arg_list,rm = Datachunk_range(rownum_sql = cnt_sql)
df = multiprocesser(arg_list)
# ===============================================================
print('all_df count : %s '% len(df))
print(df.shape)

[n_core] :  5
Data  count : 4293313
■ [PID]          : 190862 
■ [Jupyter path] : /home/cdsadmin/python_src/EY/YSR
■ [Kernel ID]    : cd80e97e-fd77-4d65-a4be-c49677bb9042

start_arg_list : [0, 858662, 1717324, 2575986, 3434648, 4293313] 

start_num : 1 | end_num : 858662
start_num : 858663 | end_num : 1717324
start_num : 1717325 | end_num : 2575986
start_num : 2575987 | end_num : 3434648
start_num : 3434649 | end_num : 4293313
all_df count : 4293313 
(4293313, 8)


In [12]:
## Mollys 중분류에서 제외되는 소동물,관상어 제외
df     = df[~df['PRDT_MCODE_CD'].isin(['0065','0132'])]
## Apriori로직에 사용될 기준 상품명으로할지,소분류로할지,중분류로할지 여기서 결정
df1    = df.groupby('CUST_ID')['PRDT_NM'].apply(list).reset_index(name='PRDT_LIST')
te     = TransactionEncoder()
te_ary = te.fit(df1['PRDT_LIST'].values).transform(df1['PRDT_LIST'].values)
te_df  = pd.DataFrame(te_ary, columns=te.columns_)

In [154]:
df1

,CUST_ID,PRDT_LIST
0,C10000281,"[반려견맛있는간식, 반려견놀이간식, 세정용품, 반려견놀이간식, 세정용품]"
1,C10000576,"[반려견놀이간식, 반려견맛있는간식, 반려견맛있는간식, 반려견맛있는간식]"
2,C10000643,"[반려견패드, 반려견놀이간식, 반려견놀이간식]"
3,C10000754,[이미용품]
4,C10000803,"[반려견이코노미, 반려견이코노미, 이미용품, 반려견이코노미, 세정용품, 세정용품, ..."
...,...,...
445525,C92587699,[묘장난감]
445526,C92587733,"[반려견맛있는간식, 반려견맛있는간식, 반려묘간식, 반려묘간식, 반려묘사료, 반려묘간..."
445527,C92587747,"[반려견교육간식, 반려견놀이간식]"
445528,C92587786,"[반려견습식사료, 반려견습식사료]"


#### Mollys데이터의 Apriori만 계산해볼경우 그냥 해당로직을 사용하면 되지만
#### 모든담당 데이터를 포함하여 연산해야하는 경우 FPGrowth 로직을 사용할 것은 추천드립니다.
#### 트리기반의 연산처리로 불필요한 연산량을 줄여버린 로직으로 
#### 컴퓨팅 성능과 분석결과 모두 뛰어난 로직입니다 ^^

In [ ]:
frequent_itemsets = apriori(te_df, min_support=0.01, use_colnames=True)
frequent_itemsets


#####   **Automatically set to 'support' if `support_only=True`.**
#####   Otherwise, supported metrics are ['support', 'confidence', 'lift', 'leverage', and 'conviction']
#####   These metrics are computed as follows:
#####   - support(A->C) = support(A+C) [aka 'support'],  range: [0, 1]
#####   - confidence(A->C) = support(A+C) / support(A), range: [0, 1]
#####   - lift(A->C) = confidence(A->C) / support(C),         range: [0, inf]
#####   - leverage(A->C) = support(A->C) - support(A)*support(C),  range: [-1, 1]
#####   - conviction = [1 - support(C)] / [1 - confidence(A->C)], range: [0, inf]


In [153]:
association_rules(frequent_itemsets, metric="support", min_threshold=0.01)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(놀이용품),(반려견건강간식),0.113743,0.137914,0.022499,0.197806,1.434264,0.006812,1.074659
1,(반려견건강간식),(놀이용품),0.137914,0.113743,0.022499,0.163138,1.434264,0.006812,1.059024
2,(놀이용품),(반려견교육간식),0.113743,0.125967,0.016899,0.148571,1.179448,0.002571,1.026549
3,(반려견교육간식),(놀이용품),0.125967,0.113743,0.016899,0.134154,1.179448,0.002571,1.023573
4,(놀이용품),(반려견놀이간식),0.113743,0.188425,0.029693,0.261051,1.385435,0.008261,1.098282
5,(반려견놀이간식),(놀이용품),0.188425,0.113743,0.029693,0.157584,1.385435,0.008261,1.052041
6,(놀이용품),(반려견맛있는간식),0.113743,0.312713,0.038186,0.335721,1.073576,0.002617,1.034636
7,(반려견맛있는간식),(놀이용품),0.312713,0.113743,0.038186,0.122112,1.073576,0.002617,1.009533
8,(놀이용품),(반려견습식사료),0.113743,0.072137,0.011570,0.101725,1.410168,0.003365,1.032939
9,(반려견습식사료),(놀이용품),0.072137,0.113743,0.011570,0.160397,1.410168,0.003365,1.055567
